<a href="https://colab.research.google.com/github/AfricArxiv/hub-and-search-portal/blob/master/osf_africarxiv_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import requests
import json
import pandas as pd

preprintApi = 'https://api.osf.io/v2/providers/preprints/africarxiv/preprints/'
preprintApi = requests.get(preprintApi)
preprintApi = json.loads(preprintApi.content)

authorsRequest = ""
papers = []

while preprintApi['links']['next']:
  for preprint in preprintApi['data']:
    paper = {}
    authorsRequest = requests.get(preprint['relationships']['contributors']['links']['related']['href'])
    authorsRequest = json.loads(authorsRequest.content)
    authorsData = authorsRequest['data']
    authorPaper = []
    authors = []

    for author in authorsData:
      if 'data' in author['embeds']['users']:
        authors.append(author['embeds']['users']['data']['attributes']['full_name'])
      elif 'errors' in author['embeds']['users']:
        authors.append(author['embeds']['users']['errors'][0]['meta']['full_name'])
      else:
        print("Something else happens")
        print(author['embeds']['users'])

    paper['authors'] = authors
    paper['doi'] = preprint['links']['preprint_doi']
    paper['link'] = preprint['links']['html']
    paper['title'] = preprint['attributes']['title']
    paper['date_published'] = preprint['attributes']['date_published']
    paper['publication_type'] = preprint['type']
    paper['abstract'] = preprint['attributes']['description']
    paper['tags'] = preprint['attributes']['tags']

    papers.append(paper)

  preprintApi = requests.get(preprintApi['links']['next'])
  preprintApi = json.loads(preprintApi.content)


In [ ]:
df = pd.DataFrame(papers)

In [ ]:
df["authors"] = df.apply(lambda x: ",".join(x["authors"]), axis=1)
df["tags"] = df.apply(lambda x: ",".join(x["tags"]), axis=1)

In [ ]:
df.to_csv('data.csv')
!cp data.csv "gdrive"